# Data Science Use Case: ASX Index Predictor Model - Part 1
# Written by: Ricky Chang

## Objectives

This series of posts describe a Data Science Use Case, which is the end-to-end process of creating a Predictor Model for ASX Indices. All code is written in Python and its machine learning libraries.

The Use Case is divided into these sections:

1. Hypothesis
1. Data Preparation
1. Feature Engineering
1. Model Selection
1. Model Validation
1. Summary of Findings

## 1) Hypothesis
### Can monthly asset prices / ratios be used as a lagging (buy) indicator for index price? Some assumptions to be validated:

* Equity stock indices reflect the overall state of the economy
* Rising non precious metal prices have a positive correlation with the overall economy
* Rising precious metal prices have a negative correlation with the overall economy, and precious metal prices

## 2) Data Preparation

Monthly time series data for various assets shall be joined together to create the initial set of features.

In [56]:
#import libraries for data preparation
import pandas as pd
import sklearn as sk
import numpy as np
import scipy

This code snippet performs these steps to prepare datasets for individual assets to be joined together:
1. Reads datasets from an Excel Spreadsheet into Pandas Dataframes.
1. Performs some standard data cleansing and column relabelling operations

The individual assets are:
1. Gold LBMA PM Fix in USD per troy ounce
1. Silver LBMA Fix in US dollars per troy ounce
1. Copper, Grade A, LME spot price, US dollars per metric ton
1. Iron Ore Fines 62% FE spot (CFR Tianjin port), US dollars per metric ton
1. Oil - West Texas Intermediate (WTI) and Brent in US dollars per barrel
1. RBA Cash Rate Target

The individual indices are:
1. All Ordinaries (XAO)
1. S&P/ASX 20 (XTL)
1. S&P/ASX 50 (XFL)

In [92]:
folder = ''
df_gold   = pd.ExcelFile(folder + 'gold-price-history.xlsx').parse('Gold History')[10:]
df_silver = pd.ExcelFile(folder + 'silver-price-history.xlsx').parse('Silver History')[10:]
df_copper = pd.ExcelFile(folder + 'copper-price-history.xlsx').parse('Copper History')[10:]
df_iron   = pd.ExcelFile(folder + 'iron-ore-price-history.xlsx').parse('Iron Ore History')[10:]
df_oil    = pd.ExcelFile(folder + 'oil-price-history.xlsx').parse('Crude Oil History')[10:]
df_xao    = pd.ExcelFile(folder + 'All Ordinaries (XAO).xlsx').parse('All Ordinaries (XAO)')[4:]
df_xtl    = pd.ExcelFile(folder + 'ASX 20 (XTL).xlsx').parse('ASX 20 (XTL).csv')[4:]
df_xfl    = pd.ExcelFile(folder + 'ASX 50 (XFL).xlsx').parse('ASX 50 (XFL).csv')[4:]
df_rba    = pd.ExcelFile(folder + 'RBA Cash Rate Target History.xlsx').parse('RBA Cash Rate Target')[6:]
df_gold.rename(columns={'Market Index':'Date','Unnamed: 1':'Gold'}, inplace=True)
df_gold.drop(columns=['Unnamed: 2','Unnamed: 3'], inplace=True)
df_silver.rename(columns={'Market Index':'Date','Unnamed: 1':'Silver'}, inplace=True)
df_copper.rename(columns={'Market Index':'Date','Unnamed: 1':'Copper'}, inplace=True)
df_iron.rename(columns={'Market Index':'Date','Unnamed: 1':'Iron'}, inplace=True)
df_oil.rename(columns={'Market Index':'Date','Unnamed: 1':'WTI','Unnamed: 2':'Brent'}, inplace=True)
df_xao.drop(columns=['Unnamed: 6','Unnamed: 7','Unnamed: 8'], inplace=True)
df_xtl = df_xtl.drop(columns=['Unnamed: 6','Unnamed: 7','Unnamed: 8'])
df_xfl = df_xfl.drop(columns=['Unnamed: 6','Unnamed: 7','Unnamed: 8'])
df_rba = df_rba.drop(columns=['Unnamed: 1','Unnamed: 3','Unnamed: 4'])
df_xao = df_xao.rename(columns=df_xao.head(1).to_dict('records')[0])[2:]
df_xtl = df_xtl.rename(columns=df_xtl.head(1).to_dict('records')[0])[2:]
df_xfl = df_xfl.rename(columns=df_xfl.head(1).to_dict('records')[0])[2:]
df_rba = df_rba.rename(columns={'RBA Cash Rate Target':'RBA Cash Rate Target Date','Unnamed: 2':'RBA Cash Rate'})
df_rba['Date'] = df_rba['RBA Cash Rate Target Date']
#df_rba.set_index('Date', inplace=True)

## Data Manipulation: Date Conversion
Dates from the indices require manipulation for the data join to be correct. Since the hypothesis uses asset prices to predict future index prices, asset prices must be ahead of index prices by at least 1 month, otherwise look-ahead bias will be introduced into the dataset.

<div class="alert alert-block alert-info">
Example:

Asset Date: 2019-04-01, which is the April 19 price.

Index Date: 2019-05-31, which is the May 19 price.

Asset April 19 price is used to predict Index May 19 price.
</div>

In order for both prices to be on the same record, their dates need to be normalised with respect to the month of the target (Index price).

Index Date must be converted from 2019-05-31 to 2019-05-01.

Asset Date must be converted from 2019-04-01 to 2019-05-01.

These functions shall be applied to the dates:

`get_first_day_of_current_month(Index Date)`

`get_first_day_of_next_month(Asset Date)`

In [87]:
import datetime as dt
def get_first_day_of_current_month(x):
    m = x.month
    y = x.year
    return dt.datetime(y,m,1)
def get_first_day_of_next_month(x):
    m = x.month
    y = x.year
    if m == 12:
        m = 1
        y += 1
    else:
        m = m + 1
    return dt.datetime(y,m,1)

This code snippet performs these steps:
1. Applies get_first_day_of_next_month() to all Dates of the individual assts
1. Sets index for individual assets
1. Joins all individual assets by index
1. Drops all NA records

In [93]:
dfs = [df_gold, df_silver, df_copper, df_iron, df_oil, df_rba]
for df in dfs:
    list_dates = []
    for i in range(df['Date'].shape[0]):
        list_dates.append(get_first_day_of_next_month(pd.to_datetime(df['Date'].values[i])))
    df['Date'] = list_dates
    
df_gold.set_index('Date', inplace=True)
df_silver.set_index('Date', inplace=True)
df_copper.set_index('Date', inplace=True)
df_iron.set_index('Date', inplace=True)
df_oil.set_index('Date', inplace=True)
df_xao.set_index('Date', inplace=True)
df_xtl.set_index('Date', inplace=True)
df_xfl.set_index('Date', inplace=True)
df_rba.set_index('Date', inplace=True)
df_asset = df_gold.join(df_silver).join(df_iron).join(df_copper).join(df_oil).join(df_rba).dropna()

In [ ]:
This code snippet performs these steps:
1. Sets index for individual indices
1. Joins all individual indices by index
1. Applies get_first_day_of_current_month() to all Dates of the joined indices
1. Joins joined assets to joined indices
1. Drops all NA records

In [94]:
df_indices = df_xao.join(df_xtl, rsuffix='_XTL').join(df_xfl, rsuffix='_XFL').dropna()
df_indices = df_indices.reset_index()
df_indices['Date_End'] = df_indices['Date']
list_datetimes = []
for i in range(df_indices['Date'].shape[0]):
    list_datetimes.append(get_first_day_of_current_month(pd.to_datetime(df_indices['Date'].values[i])))
df_indices['Date'] = list_datetimes
df_indices = df_indices.set_index('Date')
df_asset = df_asset.join(df_indices).dropna()

## The final joined raw dataset

In [95]:
df_asset

,Gold,Silver,Iron,Copper,WTI,Brent,RBA Cash Rate Target Date,RBA Cash Rate,Open,High,...,High_XTL,Low_XTL,Close_XTL,Return (%)_XTL,Open_XFL,High_XFL,Low_XFL,Close_XFL,Return (%)_XFL,Date_End
Date,,,,,,,,,,,,,,,,,,,,,
1990-03-01,416.81,5.29,14.05,2358.94,22.12,19.93,1990-02-15 00:00:00,17.5,1575.2,1604.5,...,819.21,791.83,791.83,-1.4904,1509.74,1539.55,1484.75,1484.75,-1.83147,1990-03-30
1990-04-01,393.06,5.08,14.05,2625.7,20.41,18.37,1990-03-07 00:00:00,17,1535.7,1535.7,...,788.49,725.01,725.01,-8.43868,1453.91,1478.01,1381.73,1381.73,-6.93854,1990-04-30
1990-05-01,374.24,5.06,14.05,2685.23,18.43,16.49,1990-04-04 00:00:00,17,1434.5,1517.2,...,785.9,733.6,775.76,6.9999,1391.95,1476.79,1391.95,1466.68,6.14809,1990-05-31
1990-06-01,369.05,5.07,14.05,2740.34,18.25,16.34,1990-05-02 00:00:00,15.5,1512.1,1535.5,...,786.57,767.21,772.07,-0.475663,1469.06,1487.33,1455.34,1462.83,-0.262498,1990-06-29
1990-07-01,352.33,4.92,14.05,2583.81,16.69,15.05,1990-06-06 00:00:00,15.5,1500.7,1624.6,...,862.33,770.61,830.53,7.57185,1457.95,1598.38,1457.95,1542.11,5.41963,1990-07-31
1990-08-01,362.53,4.87,14.05,2769,18.42,17.14,1990-07-04 00:00:00,15.5,1573.8,1635.9,...,857.3,763.19,787.06,-5.23401,1556.94,1586.26,1417.49,1464.94,-5.00418,1990-08-31
1990-09-01,394.73,5,14.05,2956.4,27.23,27.32,1990-08-02 00:00:00,15.5,1507.6,1513.6,...,798.05,719.83,725.9,-7.77069,1462.25,1475.5,1333.52,1342.24,-8.37577,1990-09-28
1990-10-01,389.32,4.8,14.05,3040.17,33.8,35.03,1990-09-05 00:00:00,14,1397.5,1408.2,...,735.6,681.66,695.34,-4.20995,1332.37,1359.59,1270.06,1285.49,-4.22801,1990-10-31
1990-11-01,380.74,4.39,14.05,2742.55,36.13,36.09,1990-10-15 00:00:00,14,1327.2,1377.7,...,740.24,680.34,702.31,1.00239,1262.16,1348.9,1253.57,1291.38,0.458191,1990-11-30


## Write the dataset to CSV

In [96]:
df_asset.to_csv(path_or_buf=folder+'df_asset.csv')

## Conclusion of Part 1
With the raw dataset written to a CSV, it is now ready for Feature Engineering, which will be covered in Part 2 of this series.